<a href="https://colab.research.google.com/github/alexik777/alexik777/blob/main/My1st_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf
import re

In [37]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/IMDB/IMDB Dataset.csv')
df.shape

(50000, 2)

In [57]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production br br the filmin...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there s a family where a little boy ...,negative
4,petter mattei s love in the time of money is a...,positive


In [59]:
pd.factorize(y_train)[0]

array([0, 0, 0, ..., 1, 1, 1])

In [38]:
def clean(text):
    text = text.lower()
    # text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'\W+',' ',text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'<.*?>',' ', text)
    return text


In [39]:
# text = tweets['review']
df['review'] = df['review'].apply(clean)
test_idx = np.random.randint(0, high=len(df), size=10000)
train = df.drop(test_idx, axis=0)
test = df.loc[test_idx]
X_train, y_train = train.pop('review'), train.pop('sentiment')
X_test, y_test = test.pop('review'), test.pop('sentiment')

In [58]:
y_train

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49994    negative
49995    positive
49996    negative
49998    negative
49999    negative
Name: sentiment, Length: 40930, dtype: object

In [41]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(df.iloc[:, 0].values)

In [13]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [42]:
BATCH_SIZE = 64

In [43]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in',
       'i', 'this', 'that', 's', 'was', 'as', 'movie', 'for', 'with'],
      dtype='<U14')

In [44]:
model = tf.keras.Sequential([encoder,
                             tf.keras.layers.Embedding(
                                 input_dim=len(encoder.get_vocabulary()),
                                 output_dim=64,
                                 # Use masking to handle the variable sequence lengths
                                 mask_zero=True
                             ),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=.3, return_sequences = True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(64, activation='relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1)
                            ])

In [45]:
# predict on a sample text without padding.

sample_text = ('The movie was cool')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.00224621]


In [46]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_2 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, None, 64)          64000     
                                                                 
 bidirectional_3 (Bidirectio  (None, None, 128)        66048     
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                      

In [49]:
history = model.fit(X_train.values, pd.factorize(y_train)[0], epochs=10,
                    validation_data=(X_test.values, pd.factorize(y_test)[0]),  
                    validation_steps=30)

Epoch 1/10
1280/1280 [==============================] - 469s 367ms/step - loss: 0.7196 - accuracy: 0.5558 - val_loss: 0.5729 - val_accuracy: 0.7634
Epoch 2/10
1280/1280 [==============================] - 462s 361ms/step - loss: 0.5626 - accuracy: 0.7431 - val_loss: 0.4063 - val_accuracy: 0.8416
Epoch 3/10
1280/1280 [==============================] - 464s 363ms/step - loss: 0.5842 - accuracy: 0.7962 - val_loss: 0.4948 - val_accuracy: 0.8354
Epoch 4/10
1280/1280 [==============================] - 464s 363ms/step - loss: 0.5490 - accuracy: 0.7661 - val_loss: 0.6418 - val_accuracy: 0.6015
Epoch 5/10
1280/1280 [==============================] - 471s 368ms/step - loss: 0.5010 - accuracy: 0.7752 - val_loss: 0.4778 - val_accuracy: 0.8218
Epoch 6/10
1280/1280 [==============================] - 473s 370ms/step - loss: 0.4753 - accuracy: 0.7934 - val_loss: 0.4275 - val_accuracy: 0.8283
Epoch 7/10
1280/1280 [==============================] - 471s 368ms/step - loss: 0.4959 - accuracy: 0.7830 - val_

In [50]:
test_loss, test_acc = model.evaluate(X_test.values, pd.factorize(y_test)[0])

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))


313/313 [==============================] - 61s 194ms/step - loss: 0.4046 - accuracy: 0.8455
Test Loss: 0.4046238660812378
Test Accuracy: 0.8454999923706055


In [62]:
sample_text = ('Excellent movie')
predictions = model.predict(np.array([sample_text]))

In [63]:
print(predictions)

[[0.06142167]]
